#### Este notebook esta dedicado al entrenamiendo de un modelo de serie de tiempo que nos ayude a predecir el comportamiendo de las tasas de fecundidad en Sudamérica.

In [1]:
# Importamos las librerias necesaria para cargar los datos 
import pandas as pd
import numpy as np

In [2]:
# Lista de países
countries = ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 
        'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela, RB']

# Crear DataFrames individuales para cada país usando globals()
for country in countries:
        file_path = f'../Data/Fertility_{country}.csv'
        globals()[f"{country}_data"] = pd.read_csv(file_path, usecols=['Year', 'Country Name', 'Fertility Rate'])

In [3]:
# Cargamos los datos de Sudamerica 

Sudamerica_data= pd.read_csv('../Data/Fertility_Sudamerica.csv')

#### Ya teniendo los datos cargados, comenzamos con el proceso del modelo de serie de tiempo

#### Para poder haccer un modelo de series de tiempo hay ciertas caracteristicas que son fundamentales y una de ellas es, la estacionaridad, esto quiere decir que es estable a lo largo del tiempo, es decir, cuando la media y varianza son constantes en el tiempo. 

#### Para saber que si una serie es estacionaria se puede aplicar el test de Dickey-Fuller, el cual podemos encontrar en la libreria de statsmodels, el cual nos dice, que si el p-value mos da menor que la significancia la serie es estacionaria, de lo contrario tiene problema de raiz unitaria.

In [4]:
from statsmodels.tsa.stattools import adfuller

# Realizar el test de Dickey-Fuller
result = adfuller(Sudamerica_data['Fertility Rate'])

# Extraer los resultados
adf_statistic = result[0]
p_value = result[1]
critical_values = result[4]

# Imprimir los resultados
print('ADF Statistic:', adf_statistic)
print('p-value:', p_value)
print('Critical Values:', critical_values)

ADF Statistic: -2.622699239693846
p-value: 0.08839595809490125
Critical Values: {'1%': -3.60098336718852, '5%': -2.9351348158036012, '10%': -2.6059629803688282}


#### Como vemos el p-value no es menor que 0.05, por lo que es posible necesiten más análisis o transformaciones para lograr la estacionalidad deseada en la serie temporal.

#### Pero es normal que estos resultados sean de esta forma ya que si vimos en el analisis (Ver [aquí](https://github.com/JuankTS/Fecuendidad-en-America-Latina-/blob/main/Analisis_data/Analisis.ipynb)) las series no muestran estas caracteristicas de media y varianza constante, para casos como este se usar otro tipos de modelos como, lo puede ser el modelo de suavizado exponencial, que es un metodo que lo que hace es darle más peso a las obervaciones más recientes y menos peso a las más antiguas ([Más información aquí](https://www.ibm.com/docs/es/spss-statistics/saas?topic=modeler-custom-exponential-smoothing-models)), esto para tener encuenta los valores de las opciones más recientes en el pronostico o predicción futura.

In [9]:
# Usamos el modelo de Holt con tendencia aditiva ya que las tasas de fecundidad vienen con una tendencia a la baja
# Importamos el modelo 
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Iplementamos los datos al modelo y entremanamos el modelo
model = ExponentialSmoothing(Sudamerica_data['Fertility Rate'], trend='add')
fit = model.fit()

# Hacemos un pronostico para los sigientes 6 años 
forecast = fit.forecast(steps=8)
future_years = list(range(2023, 2031))

In [10]:
# Importamos la librerias para el grafico 
import plotly.express as px
import plotly.graph_objects as go

# Crear el gráfico con Plotly Express
fig = px.line(Sudamerica_data, x='Year', y='Fertility Rate', title='Pronóstico de Tasa de Fertilidad usando Holt (Tendencia Aditiva)')

# Añadir los datos pronosticados al gráfico directamente
fig.add_trace(go.Scatter(x=future_years, y=forecast,
                        mode='lines+markers', name='Pronóstico Holt',
                        line=dict(dash='dash')))

# Añadir etiquetas y leyenda
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Fertility Rate',
    legend_title_text='Series'
)

# Mostrar el gráfico
fig.show()